In [11]:
import os
import subprocess


# 要執行的命令及參數

# 所有照片訓練結果
# command = ["python", "./detect.py","--weights", "runs/train/exp/AllData.pt","--source","data/images", "--save-txt"]

# 二次訓練第一次訓練結果
# command = ["python", "./detect.py","--weights", "runs/train/exp/FirstTrain.pt","--source","data/images", "--save-txt"]

# 二次訓練第二次訓練結果
command = ["python", "./detect.py","--weights", "runs/train/exp/SecoundTrain.pt","--source","data/images", "--save-txt"]

# 使用 subprocess 模組執行命令
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 等待命令執行完成
stdout, stderr = process.communicate()

# 檢查執行結果
if process.returncode == 0:
    print("執行成功！")
    print("輸出：", stdout.decode())
else:
    print("執行失敗！")
    print("錯誤信息：", stderr.decode())

# 定義結果保存的根目錄
results_dir = "runs/detect/"

# 列出目錄中的所有文件和文件夾
contents = os.listdir(results_dir)

# 找到最新生成的目錄
latest_exp_dir = max(contents, key=lambda x: os.path.getctime(os.path.join(results_dir, x)))

print(latest_exp_dir)

執行成功！
輸出： 
exp28


In [1]:
import numpy as np

# 讀取YOLOv5的txt文件，格式：class, x_center, y_center, width, height (normalized)
bounding_boxes = []
with open('C:/Users/user/Desktop/yolofloor/runs/detect/exp48/labels/1.txt', 'r') as file:
    for line in file:
        data = list(map(float, line.split()))
        bounding_boxes.append(data)

# 設定Y軸方向的閾值，這個值可以根據實際情況調整，來確定哪些box是在同一樓層
y_threshold = 0.1

# 將box按y_center排序
bounding_boxes = sorted(bounding_boxes, key=lambda x: x[2])

merged_boxes = []
current_floor = []

# 遍歷所有bounding box，根據y_center的差值來判斷是否屬於同一樓層
for box in bounding_boxes:
    if not current_floor:
        current_floor.append(box)
    else:
        # 判斷是否同一樓層（根據y_center是否接近）
        if abs(box[2] - current_floor[-1][2]) < y_threshold:
            current_floor.append(box)
        else:
            # 合併同一樓層的Bounding Box
            x_centers = [b[1] for b in current_floor]
            y_centers = [b[2] for b in current_floor]
            widths = [b[3] for b in current_floor]
            heights = [b[4] for b in current_floor]
            
            # 計算合併後的bounding box
            min_x = min(x - w/2 for x, w in zip(x_centers, widths))
            max_x = max(x + w/2 for x, w in zip(x_centers, widths))
            min_y = min(y - h/2 for y, h in zip(y_centers, heights))
            max_y = max(y + h/2 for y, h in zip(y_centers, heights))
            
            # 合併後的中心點與寬高
            merged_x_center = (min_x + max_x) / 2
            merged_y_center = (min_y + max_y) / 2
            merged_width = max_x - min_x
            merged_height = max_y - min_y
            
            merged_boxes.append([0, merged_x_center, merged_y_center, merged_width, merged_height])
            
            # 開始處理下一樓層
            current_floor = [box]

# 處理最後一層
if current_floor:
    x_centers = [b[1] for b in current_floor]
    y_centers = [b[2] for b in current_floor]
    widths = [b[3] for b in current_floor]
    heights = [b[4] for b in current_floor]

    min_x = min(x - w/2 for x, w in zip(x_centers, widths))
    max_x = max(x + w/2 for x, w in zip(x_centers, widths))
    min_y = min(y - h/2 for y, h in zip(y_centers, heights))
    max_y = max(y + h/2 for y, h in zip(y_centers, heights))

    merged_x_center = (min_x + max_x) / 2
    merged_y_center = (min_y + max_y) / 2
    merged_width = max_x - min_x
    merged_height = max_y - min_y

    merged_boxes.append([0, merged_x_center, merged_y_center, merged_width, merged_height])

# 輸出結果
for box in merged_boxes:
    print(box)


[0, 0.37109325, 0.21289075000000002, 0.7187495, 0.20703150000000003]
[0, 0.4082035, 0.3828125, 0.785157, 0.19531300000000007]
[0, 0.208984, 0.560547, 0.3867189999999999, 0.24609400000000003]
[0, 0.48828125, 0.804688, 0.9453134999999999, 0.30468800000000007]


In [12]:
import glob
import pandas as pd

# 使用 os.path.join 組合路徑
# folder_path = os.path.join("./runs", "detect", "exp22", "labels")

folder_path = os.path.join("./runs", "detect", latest_exp_dir, "labels") #用於自動化

search_path = os.path.join(folder_path, "*.txt")

# 列出所有 .txt 文件
all_txt_files = glob.glob(search_path)

# 創建一個空的 DataFrame，來保存結果
data = {
    "txt_file": [],
    "bounding_box_count": [],
    "y_group_count": [],
    "x_group_count": []
}

for txt_file in all_txt_files:
    with open(txt_file, "r") as file:
        lines = file.readlines()

    print(txt_file)
    # 儲存 bounding box 的 x_center 座標
    x_centers = []
    y_centers = []

    # 解析每一行標註
    for line in lines:
        class_label, x_center, y_center, width, height = map(float, line.split())

        x_centers.append(x_center)
        y_centers.append(y_center)

    # 計算 Bounding Box 的數量
    bounding_box_count = len(lines)
    
    # 將 y_center 座標排序
    y_centers.sort()
    x_centers.sort()

    # 初始化計數器
    x_group_count = 0
    y_group_count = 0

    x_group_tolerance = 0.2  # x分組容忍度
    y_group_tolerance = 0.02  # y分組容忍度

    x_current_group = [x_centers[0]]  # 目前的分組
    y_current_group = [y_centers[0]]  # 目前的分組

    # 檢查 x_center 座標是否接近彼此的 bounding box
    for i in range(1, len(x_centers)):
        if abs(x_centers[i] - x_current_group[-1]) <= x_group_tolerance:
            x_current_group.append(x_centers[i])  # 將 x_center 加入目前的分組
        else:
            x_group_count += 1

            # print(f"Group {x_group_count}: {x_current_group}")  # 印出目前的分組

            x_current_group = [x_centers[i]]  # 開始新的分組

    # 印出最後一組
    if x_current_group:
        x_group_count += 1
        
        # print(f"Group {x_group_count}: {x_current_group}")

    # print("共有", x_group_count, "組 bounding box 的 x_center 座標接近。")

    # 檢查 y_center 座標是否接近彼此的 bounding box
    for i in range(1, len(y_centers)):
        if abs(y_centers[i] - y_current_group[-1]) <= y_group_tolerance:
            y_current_group.append(y_centers[i])  # 將 x_center 加入目前的分組
        else:
            y_group_count += 1
            
            # print(f"Group {y_group_count}: {y_current_group}")  # 印出目前的分組
            
            y_current_group = [y_centers[i]]  # 開始新的分組

    # 印出最後一組
    if y_current_group:
        y_group_count += 1
        
    # 將結果保存到 data 字典中
    
    data["txt_file"].append(txt_file)
    data["bounding_box_count"].append(bounding_box_count)
    data["y_group_count"].append(y_group_count)
    data["x_group_count"].append(x_group_count)
    
    # 將結果存儲到 DataFrame 中
    df = pd.DataFrame(data)

    # 將 DataFrame 輸出為 Excel 文件
    output_path = os.path.join("./runs", "detect","building_analysis_DatasetsSecound.xlsx")
    df.to_excel(output_path, index=False)

    print("此建築為",y_group_count,"層",x_group_count,"跨")

./runs\detect\exp28\labels\1.txt
此建築為 4 層 2 跨
./runs\detect\exp28\labels\138.txt
此建築為 4 層 3 跨
./runs\detect\exp28\labels\142.txt
此建築為 6 層 1 跨
./runs\detect\exp28\labels\145.txt
此建築為 9 層 2 跨
./runs\detect\exp28\labels\15.txt
此建築為 4 層 3 跨
./runs\detect\exp28\labels\16.txt
此建築為 4 層 3 跨
./runs\detect\exp28\labels\168.txt
此建築為 6 層 1 跨
./runs\detect\exp28\labels\182.txt
此建築為 8 層 1 跨
./runs\detect\exp28\labels\192.txt
此建築為 13 層 1 跨
./runs\detect\exp28\labels\194.txt
此建築為 11 層 1 跨
./runs\detect\exp28\labels\196.txt
此建築為 10 層 1 跨
./runs\detect\exp28\labels\2.txt
此建築為 5 層 3 跨
./runs\detect\exp28\labels\204.txt
此建築為 6 層 1 跨
./runs\detect\exp28\labels\207.txt
此建築為 7 層 1 跨
./runs\detect\exp28\labels\212.txt
此建築為 7 層 1 跨
./runs\detect\exp28\labels\214.txt
此建築為 7 層 3 跨
./runs\detect\exp28\labels\219.txt
此建築為 4 層 1 跨
./runs\detect\exp28\labels\220.txt
此建築為 8 層 1 跨
./runs\detect\exp28\labels\226.txt
此建築為 8 層 1 跨
./runs\detect\exp28\labels\229.txt
此建築為 6 層 1 跨
./runs\detect\exp28\labels\238.txt
此建築為 2 層

In [3]:
import pandas as pd

# 读取 Excel 文件
file_path = r"C:\\Users\\user\\Desktop\\Data_Base_kun\\yolv5樓層數結果.xlsx"

# 使用 pandas 读取目标工作表的数据
df = pd.read_excel(file_path, sheet_name='1')

df.iloc[0, 0] = y_group_count  # 修改楼层数 (B2, 注意 iloc 是从 0 开始计数)
df.iloc[0, 1] = x_group_count  # 修改跨数 (B3)

# 保存修改后的 DataFrame 到 Excel 文件
df.to_excel(file_path, sheet_name='1', index=False)

print("Excel 文件更新成功！")


Excel 文件更新成功！


In [15]:
import subprocess

# 要執行的命令及參數
command = ["python", "C:/Users/user/Desktop/Data_Base_kun/Python_MainProgram.py"]

# 指定工作目录为其他文件夹的路径
working_directory = "C:/Users/user/Desktop/Data_Base_kun/"

# 使用 subprocess 模块执行命令，并指定工作目录
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

# 等待命令執行完成
stdout, stderr = process.communicate()

# 檢查執行結果
if process.returncode == 0:
    print("執行成功！")
    print("輸出：", stdout.decode())
else:
    print("執行失敗！")
    try:
        print("错误信息：", stderr.decode(errors='ignore'))  # 忽略无法解码的字节
    except Exception as e:
        print("无法解码错误信息：", e)

執行失敗！
错误信息： c:\Users\user\anaconda3\envs\YoloV5\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\user\anaconda3\envs\YoloV5\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\user\anaconda3\envs\YoloV5\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
findfont: Generic family 'sans-se